<a href="https://colab.research.google.com/github/pragyan2905/Skin-Cancer-Detection/blob/main/skin_cancer_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle-4.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!kaggle datasets download -d jaiahuja/skin-cancer-detection -p /content


Dataset URL: https://www.kaggle.com/datasets/jaiahuja/skin-cancer-detection
License(s): CC0-1.0
 93% 727M/786M [00:03<00:00, 207MB/s]
100% 786M/786M [00:03<00:00, 231MB/s]


In [5]:
import zipfile
with zipfile.ZipFile('/content/skin-cancer-detection.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/skin_cancer_dataset')
!ls /content/skin_cancer_dataset


'Skin cancer ISIC The International Skin Imaging Collaboration'


In [22]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

In [10]:
data_dir = '/content/skin_cancer_dataset/Skin cancer ISIC The International Skin Imaging Collaboration'
train_path = os.path.join(data_dir, 'Train')
test_path = os.path.join(data_dir, 'Test')


In [11]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [13]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


Found 2239 files belonging to 9 classes.
Using 1792 files for training.
Found 2239 files belonging to 9 classes.
Using 447 files for validation.


In [14]:
class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print("Class Names:", class_names)

Class Names: ['actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus', 'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']


In [23]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

x = Flatten()(base_model.output)

x = Dense(128, activation='relu')(x)

x = Dropout(0.5)(x)

output = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [24]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [25]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer_2[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 12,079,020 (46.08 MB)

 Trainable params: 8,029,449 (30.63 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [21]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

data_augmentation = Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.2),
  layers.RandomZoom(0.2),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [26]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopper = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_ds,
    epochs=30,
    validation_data=val_ds,
    callbacks=[early_stopper]
)

Epoch 1/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 76s 834ms/step - accuracy: 0.2251 - loss: 7.1055 - val_accuracy: 0.2931 - val_loss: 2.0242
Epoch 2/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 44s 510ms/step - accuracy: 0.2698 - loss: 2.0461 - val_accuracy: 0.3512 - val_loss: 2.0109
Epoch 3/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 41s 512ms/step - accuracy: 0.3048 - loss: 2.0120 - val_accuracy: 0.3266 - val_loss: 1.9295
Epoch 4/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 41s 511ms/step - accuracy: 0.2908 - loss: 1.9601 - val_accuracy: 0.3647 - val_loss: 1.8968
Epoch 5/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 29s 514ms/step - accuracy: 0.3373 - loss: 1.8993 - val_accuracy: 0.3758 - val_loss: 1.7716
Epoch 6/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 41s 511ms/step - accuracy: 0.3125 - loss: 1.8322 - val_accuracy: 0.3714 - val_loss: 1.8368
Epoch 7/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 30s 536ms/step - accuracy: 0.3004 - loss: 1.8851 - val_accuracy: 0.3960 - val_loss: 1.8438
Epoch 8/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 40s 519ms/step - accuracy: 0.3407 - loss: 1.8262 - val_accu

In [27]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

fine_tune_epochs = 10
total_epochs = 15 + fine_tune_epochs

history_fine = model.fit(
    train_ds,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=val_ds,
    callbacks=[early_stopper]
)

Epoch 13/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 61s 726ms/step - accuracy: 0.3776 - loss: 1.6934 - val_accuracy: 0.4161 - val_loss: 1.7098
Epoch 14/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 30s 526ms/step - accuracy: 0.3624 - loss: 1.7312 - val_accuracy: 0.4094 - val_loss: 1.7116
Epoch 15/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 40s 518ms/step - accuracy: 0.3840 - loss: 1.6655 - val_accuracy: 0.4072 - val_loss: 1.7145
Epoch 16/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 44s 561ms/step - accuracy: 0.3475 - loss: 1.7292 - val_accuracy: 0.4027 - val_loss: 1.7185
